In [ ]:
from javascript import require, On, Once, AsyncTask, once, off, globalThis
import json
import llm
import logging
import time

MINECRAFT_VERSION = '1.20.2'

collectDroppedItem = require('./control_primitives/collectDroppedItem.js')
craft = require('./control_primitives/craftItem.js')
explore = require('./control_primitives/exploreUntil.js')
move = require('./control_primitives/goToPosition.js')
kill = require('./control_primitives/killMob.js')
minePosition = require('./control_primitives/mineBlockPosition.js')
mineType = require('./control_primitives/mineBlockType.js')
place = require('./control_primitives/placeItem.js')
shoot = require('./control_primitives/shoot.js')
smelt = require('./control_primitives/smeltItem.js')
chest = require('./control_primitives/useChest.js')

Vec3 = require('vec3').Vec3
mineflayer = require('mineflayer')
pathfinder = require('mineflayer-pathfinder')
pvp = require('mineflayer-pvp').plugin
mcData = require('minecraft-data')(MINECRAFT_VERSION)
armorManager = require("mineflayer-armor-manager")
autoeat = require('mineflayer-auto-eat').plugin
collectblock = require('mineflayer-collectblock').plugin
hawkeye = require('minecrafthawkeye')
toolPlugin = require('mineflayer-tool').plugin
mineflayerViewer = require('prismarine-viewer')

RANGE_GOAL = 1

# Load bot
def load_bot(username=None):
  # Create bot
  if not username:
    random_number = 176
    BOT_USERNAME = f'poo_bucket{random_number}'

  bot = mineflayer.createBot({ 
      'host': 'localhost',
      'port': 22222,
      'username': BOT_USERNAME, 
      'hideErrors': False 
  })

  once(bot, 'login')
  bot.chat('I spawned')
  #   mineflayerViewer(bot, { 'port': 3000, 'firstPerson': True })

  # Load plugins
  bot.loadPlugin(collectblock)
  bot.loadPlugin(hawkeye)
  bot.loadPlugin(pathfinder.pathfinder)
  bot.loadPlugin(pvp)
  bot.loadPlugin(toolPlugin)

  globalThis.mcData = mcData
  globalThis.pathfinder = pathfinder
  movements = pathfinder.Movements(bot, mcData)
  bot.pathfinder.setMovements(movements)

  # Auto armor
  bot.loadPlugin(armorManager)
  bot.armorManager.equipAll()

  # Auto eat
  bot.loadPlugin(autoeat)
  bot.autoEat.options.priority = 'foodPoints'
  bot.autoEat.options.startAt = 14
  bot.autoEat.options.bannedFood.push('golden_apple')

  @On(bot, 'chat')
  def handleMsg(this, sender, message, *args):
      if sender and (sender != bot.username):
          bot.chat('Hi, you said ' + message)
          if 'come' in message:
              player = bot.players[sender]
              target = player.entity
              if not target:
                  bot.chat("I don't see you !")
                  return
              pos = target.position
              bot.pathfinder.setMovements(movements)
              bot.pathfinder.setGoal(pathfinder.goals.GoalNear(pos.x, pos.y, pos.z, RANGE_GOAL))
          if 'stop' in message:
              off(bot, 'chat', handleMsg)

  return bot, movements

In [ ]:
bot, movements = load_bot()

In [ ]:
CHESTS = [mcData.blocksByName[name]['id'] for name in mcData.blocksByName if 'chest' in name]
BEDS = [mcData.blocksByName[name]['id'] for name in mcData.blocksByName if 'bed' in name]

def getEnvironmentInfo(bot):
    def distanceTo(pos1, pos2):
        return ((pos1.x - pos2.x) ** 2 + (pos1.y - pos2.y) ** 2 + (pos1.z - pos2.z) ** 2) ** 0.5

    def nearbyEntities(bot, max_distance={'mobs': 32, 'items': 16, 'players': 128}):
        nearby_mobs, nearby_droppeditems, nearby_players = [], [], []
        for entity in bot.entities:
            if bot.entities[entity]:
                # Mob
                if bot.entities[entity].type not in ['other', 'player', 'projectile'] and distanceTo(bot.entities[entity].position, bot.entity.position) < max_distance['mobs']:
                    if bot.canSeeBlock(bot.blockAt(bot.entities[entity].position)):
                        nearby_mobs.append(f'{bot.entities[entity].type} {bot.entities[entity].name} is {int(distanceTo(bot.entities[entity].position, bot.entity.position))} blocks away.')
                # Item
                elif bot.entities[entity].displayName == 'Item' and distanceTo(bot.entities[entity].position, bot.entity.position) < max_distance['items']:
                    if bot.canSeeBlock(bot.blockAt(bot.entities[entity].position)):

                        nearby_droppeditems.append(f'DroppedItem {mcData.items[bot.entities[entity].metadata[8].itemId].displayName} is {int(distanceTo(bot.entities[entity].position, bot.entity.position))} blocks away.')
                # Player
                elif bot.entities[entity].type == 'player' and bot.entities[entity].username != bot.username and distanceTo(bot.entities[entity].position, bot.entity.position) < max_distance['players']:
                    if bot.canSeeBlock(bot.blockAt(bot.entities[entity].position)):

                        nearby_players.append(f'Player {bot.entities[entity].username} is {int(distanceTo(bot.entities[entity].position, bot.entity.position))} blocks away.')
        
        return nearby_mobs, nearby_droppeditems, nearby_players


    def nearbyBeds(bot, beds, max_distance=24, count=1):
        beds = bot.findBlocks({'matching': beds, 'maxDistance': max_distance, 'count': count})
        return [f'Bed at Vec3{bed.x, bed.y, bed.z} is {int(distanceTo(bed, bot.entity.position))} blocks away.' for bed in beds]

    def nearbyChests(bot, chests, max_distance=24, count=1):
        chests = bot.findBlocks({'matching': chests, 'maxDistance': max_distance, 'count': count})
        return [f'Chest at Vec3{chest.x, chest.y, chest.z} is {int(distanceTo(chest, bot.entity.position))} blocks away.' for chest in chests]
    
    nearby_mobs, nearby_droppeditems, nearby_players = nearbyEntities(bot, max_distance={'mobs': 64, 'items': 32, 'players': 128})

    nearby_beds = nearbyBeds(bot, BEDS, 24)
    nearby_chests = nearbyChests(bot, CHESTS, 24)

    return {
        'mobs': nearby_mobs,
        'droppedItems': nearby_droppeditems,
        'players': nearby_players,
        'beds': nearby_beds,
        'chests': nearby_chests
    }

def getWorldInfo(bot):
    # Time
    time = bot.time.timeOfDay
    if 0 <= time < 9000:
        time_description = 'in the Morning'
    elif 9000 <= time < 12000:
        time_description = 'at Dusk'
    else:
        time_description = 'at Night'
    
    # Weather
    weather = 'It is clear.' 
    if bot.rainState:
        weather = 'It is raining and thundering!' if bot.thunderState else 'It is raining!'
    
    return {
        'time': f'It is currently {time} {time_description}',
        'weather': weather,
        'biome': mcData.biomes[bot.blockAt(bot.entity.position).biome.id].displayName
    }

def getPlayerState(bot):
    # Inventory
    inventory = []
    for item in bot.inventory.items():
        inventory.append(f'{item.displayName} x{item.count}')
    if len(inventory) == 0: 
        inventory = 'Empty'

    # Held Item
    heldItem = 'Nothing'
    if bot.heldItem:
        heldItem = bot.heldItem.displayName

    return {
        'position': f'Vec3{bot.entity.position.x, bot.entity.position.y, bot.entity.position.z}',
        'health': bot.health,
        'hunger': bot.food,
        'heldItem': heldItem,
        'inventory': inventory
    }

def getPromptInfo(bot):
    world_info = getWorldInfo(bot)
    environment_info = getEnvironmentInfo(bot)
    player_state = getPlayerState(bot)

    return {
        'world_info': world_info,
        'environment_info': environment_info,
        'player_state': player_state
    }

In [ ]:
with open('descriptions.json', 'r') as file:
    bot_functions = json.load(file)
    
llms = llm.get_llms()

logging.basicConfig(filename='./process_log.txt', level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')

Need: https://api.python.langchain.com/en/latest/_modules/langchain/output_parsers/openai_functions.html#

In [ ]:
logging.info('Bot is starting up...')

# Generate some Tasks and Execute them
previous_tasks, previous_actions = [], []
while True:

    attempts = 0
    while True:
        try:
            # Create a Goal and High-Level Plan
            plan = llms['planning_module'].invoke({
                'current_environment': getPromptInfo(bot),
                'previous_tasks': previous_tasks,
                'bot_functions': bot_functions,
            }).content
            print(plan)
            plan = json.loads(plan)
            break
        except:
            print('Error loading json in Planning Module...')
            attempts += 1
            if attempts == 5: break
            continue

    logging.info(f'Planning Module: {plan}')

    # We have a current goal and a high level plan to achieve it
    current_goal = plan['current_goal']
    plan = plan['high_level_steps']

    # Execute on this plan
    while True:
            
        while True:
            # Decide the next best action to accomplish this plan. If the GOAL is complete summarize what we learned
            try:
                decide_action = llms['decision_module'].invoke({
                    'current_goal': current_goal,
                    'plan': plan,
                    'current_environment': getPromptInfo(bot),
                    'previous_actions': previous_actions,
                    'bot_functions': bot_functions
                }).content
                decide_action = json.loads(decide_action)
                break
            except:
                print('Error loading json in Decision Module...')
                continue


        # decide_action = json.loads(decide_action)
        decision = decide_action['decision']

        logging.info(f'Decision Module: {decide_action}')

        # The GOAL is complete, so we summarize what we learned
        if "COMPLETE" in decision:
            summarize_module = llms['summarize_module'].invoke({
                'current_goal': current_goal,
                'plan': plan,
                'current_environment': getPromptInfo(bot),
                'previous_actions': previous_actions
            }).content


            logging.info(f'GOAL COMPLETE: {current_goal}')
            logging.info(f'Summarize Module: {summarize_module}')

            # Add our learnings to the Bot's context window, then go get the next GOAL
            previous_tasks.append(summarize_module)
            break

        # The GOAL is incomplete, so we execute on our next best action
        else:
            # Attempt to execute this action
            previous_attempts = []
            while True:
                    
                while True:
                    try:

                        action_module = llms['action_module'].invoke({
                            'current_environment': getPromptInfo(bot),
                            'current_goal': decision,
                            'previous_attempts': previous_attempts,
                            'bot_functions': bot_functions
                        }).content

                        # This should be a function that we can execute
                        action_module = json.loads(action_module)
                        break
                    except:
                        print('Error loading json in Action Module...')
                        continue
                
                action = action_module['action']

                logging.info(f'Action Module: {action_module}')
                
                # Get the environment before execution
                initial_environment = getPromptInfo(bot)
                
                # Execute the action
                try:
                    result = exec(action)
                except Exception as e:
                    # If we get an error, then we know we failed
                    print(e)
                    logging.info(f'{action} failed: {e}')
                    previous_attempts.append(f'{action} failed: {e}')
                    continue
                
                # If result = True, then our Action function completed its job, so let's summarize what we did
                if result == True:
                    # First we ensure our Bot is really done, and has stopped moving
                    timeout = 0
                    while bot.pathfinder.isMoving() and timeout < 120:
                        time.sleep(5)
                        timeout += 5
                        logging.info('Bot is still moving...')
                        if timeout == 120:
                            print('Bot may be broken...')
                            break
                    
                    # Summarize the outcome of the action
                    execution_module_success = llms['execution_module_sucess'].invoke({
                        'desired_action': decision,
                        'action_executed': action,
                        'initial_environment': initial_environment,
                        'final_environment': getPromptInfo(bot)
                    }).content                  

                    logging.info(f'Execution Module Success: {execution_module_success}')

                    # Add this summary to our decision module's context window, and move on to the next action
                    previous_actions.append(execution_module_success)
                    break

                # If result = False, then our Action function failed, so we will try again
                if result == False:
                    previous_attempts.append(f'{action} failed...')

                    # If we have tried 5 times, then we need to escalate back to the decision module
                    if len(previous_attempts) == 5:
                        execution_module_failure = llms['execution_module_failure'].invoke({
                            'desired_action': decision,
                            'failed_actions': previous_attempts,
                            'initial_environment': initial_environment,
                            'final_environment': getPromptInfo(bot)
                        }).content

                        logging.info(f'Execution Module Failure: {execution_module_failure}')

                        # Add this summary of our failures to our decision module's context window, so we can generate a better action
                        previous_actions.append(execution_module_failure)
                        break